In [1]:
%load_ext autoreload
%autoreload 2

import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import pandas as pd
import re
from datetime import timedelta
import plotly.express as px

from op_analytics.datasources.defillama.dataaccess import DefiLlama
from op_analytics.coreutils.request import get_data, new_session

import urllib3
import warnings
pd.set_option('display.float_format', lambda x: '%.3f' % x)
urllib3.disable_warnings()
warnings.filterwarnings("ignore")

In [3]:
PATTERNS_TO_FILTER = [
    "-borrowed",
    "-vesting",
    "-staking",
    "-pool2",
    "-treasury",
    "-cex",
    "^treasury$",
    "^borrowed$",
    "^staking$",
    "^pool2$",
    "^pool2$",
    "polygon-bridge-&-staking",  # Added this as a full match
    ".*-cex$",  # Added this to match anything ending with -cex
]

CATEGORIES_TO_FILTER = ["CEX", "Chain"]

alignment_dict = {
    "Metis": "OP Stack fork",
    "Blast": "OP Stack fork",
    "Mantle": "OP Stack fork",
    "Zircuit": "OP Stack fork",
    "RSS3": "OP Stack fork",
    "Rollux": "OP Stack fork",
    "Ancient8": "OP Stack fork",
    "Manta": "OP Stack fork",
    "Cyber": "OP Chain",
    "Mint": "OP Chain",
    "Ham": "OP Chain",
    "Polynomial": "OP Chain",
    "Lisk": "OP Chain",
    "BOB": "OP Chain",
    "Mode": "OP Chain",
    "World Chain": "OP Chain",
    "Base": "OP Chain",
    "Kroma": "OP Chain",
    "Boba": "OP Chain",
    "Fraxtal": "OP Chain",
    "Optimism": "OP Chain",
    "Shape": "OP Chain",
    "Zora": "OP Chain"
}

alignment_df = pd.DataFrame(list(alignment_dict.items()), columns=["chain", "alignment"])

token_data = [
    {"token": "ETH", "token_category": "Native Asset"},
    {"token": "WETH", "token_category": "Native Asset"},
    {"token": "SOL", "token_category": "Native Asset"},
    {"token": "wBTC", "token_category": "Wrapped Assets"},
    {"token": "cbBTC", "token_category": "Wrapped Assets"},
    {"token": "MBTC", "token_category": "Wrapped Assets"},

    {"token": "stETH", "token_category": "Liquid Staking"},
    {"token": "wstETH", "token_category": "Liquid Staking"},
    {"token": "eETH", "token_category": "Liquid Restaking"},
    {"token": "weETH", "token_category": "Liquid Restaking"},
    {"token": "sfrxETH", "token_category": "Liquid Staking"},
    {"token": "rETH", "token_category": "Liquid Staking"},
    {"token": "mETH", "token_category": "Liquid Staking"},
    {"token": "rsETH", "token_category": "Liquid Restaking"},
    {"token": "cbETH", "token_category": "Liquid Staking"},
    {"token": "ezETH", "token_category": "Liquid Restaking"},
    {"token": "rswETH", "token_category": "Liquid Restaking"},
    {"token": "swETH", "token_category": "Liquid Staking"},
    {"token": "frxETH", "token_category": "Liquid Staking"},
    {"token": "ETHX", "token_category": "Liquid Staking"},
    {"token": "lsETH", "token_category": "Liquid Staking"},
    {"token": "oETH", "token_category": "Liquid Staking"},
    {"token": "EBTC", "token_category": "Liquid Restaking"},
    {"token": "LBTC", "token_category": "Liquid Restaking"},
    {"token": "SUPEROETHB", "token_category": "Liquid Staking"},
    {"token": "WSUPEROETHB", "token_category": "Liquid Staking"},
    {"token": "TETH", "token_category": "Liquid Staking"},
    {"token": "OSETH", "token_category": "Liquid Staking"},
    {"token": "cmETH", "token_category": "Liquid Restaking"},
    {"token": "WRSETH", "token_category": "Liquid Restaking"},
    {"token": "WEETH.BASE", "token_category": "Liquid Restaking"},
    
    {"token": "USDC", "token_category": "Stablecoins"},
    {"token": "USDT", "token_category": "Stablecoins"},
    {"token": "FDUSD", "token_category": "Stablecoins"},
    {"token": "PYUSD", "token_category": "Stablecoins"},
    {"token": "TUSD", "token_category": "Stablecoins"},
    {"token": "DAI", "token_category": "Stablecoins"},
    {"token": "USDE", "token_category": "Stablecoins"},
    {"token": "USDD", "token_category": "Stablecoins"},
    {"token": "FRAX", "token_category": "Stablecoins"},
    {"token": "EURC", "token_category": "Stablecoins"},
    {"token": "AGEUR", "token_category": "Stablecoins"},
    {"token": "USDS", "token_category": "Stablecoins"},
    {"token": "USDB", "token_category": "Stablecoins"},
    {"token": "DOLA", "token_category": "Stablecoins"},
    {"token": "SUSDE", "token_category": "Stablecoins"},
    {"token": "USD0++", "token_category": "Stablecoins"},
    {"token": "USD0", "token_category": "Stablecoins"},
    {"token": "SUSD", "token_category": "Stablecoins"},
    {"token": "CRVUSD", "token_category": "Stablecoins"},
    {"token": "USDC+", "token_category": "Stablecoins"},
    {"token": "USDZ", "token_category": "Stablecoins"},
    {"token": "STAR", "token_category": "Stablecoins"},
    {"token": "USDBC", "token_category": "Stablecoins"},
    {"token": "USD+", "token_category": "Stablecoins"},
    {"token": "CDXUSD", "token_category": "Stablecoins"},
    {"token": "HYUSD", "token_category": "Stablecoins"},
    {"token": "STAR", "token_category": "Stablecoins"},
    {"token": "EURS", "token_category": "Stablecoins"},
    {"token": "AXLEUROC", "token_category": "Stablecoins"},


    # Solana Liquid staking
    {"token": "MSOL", "token_category": "Liquid Staking"},
    {"token": "JUPSOL", "token_category": "Liquid Staking"},
    {"token": "BNSOL", "token_category": "Liquid Staking"},
    {"token": "SSOL", "token_category": "Liquid Restaking"},
    {"token": "BBSOL", "token_category": "Liquid Restaking"},
    {"token": "LAINESOL", "token_category": "Liquid Staking"},
    {"token": "STSOL", "token_category": "Liquid Staking"},
    {"token": "STRONGSOL", "token_category": "Liquid Staking"},
    {"token": "HUBSOL", "token_category": "Liquid Staking"},
    {"token": "PATHSOL", "token_category": "Liquid Staking"},
    {"token": "STEPSOL", "token_category": "Liquid Staking"},
    {"token": "EDGESOL", "token_category": "Liquid Staking"},
    {"token": "JITOSOL", "token_category": "Liquid Staking"},
    {"token": "DSOL", "token_category": "Liquid Staking"},
    {"token": "BONKSOL", "token_category": "Liquid Staking"},
    {"token": "VSOL", "token_category": "Liquid Staking"},
    {"token": "HSOL", "token_category": "Liquid Staking"},
    # {"token": "ARB", "token_category": "Layer 2 Token"},
    # {"token": "OP", "token_category": "Layer 2 Token"},
    # {"token": "MODE", "token_category": "Layer 2 Token"},
]

token_categories = pd.DataFrame(token_data)

token_categories["token"] = token_categories["token"].str.upper()


mapping = {
    "Dexes": "Trading",
    "Liquidity manager": "Yield",
    "Derivatives": "Derivatives",
    "Yield Aggregator": "Yield",
    "Indexes": "Yield",
    "Bridge": "Trading",
    "Leveraged Farming": "Yield",
    "Cross Chain": "Trading",
    "CDP": "Lending",
    "Farm": "Yield",
    "Options": "Trading",
    "DCA Tools": "Trading",
    "Services": "TradFi/Fintech",
    "Chain": "TradFi/Fintech",
    "Privacy": "TradFi/Fintech",
    "RWA": "TradFi/Fintech",
    "Payments": "TradFi/Fintech",
    "Launchpad": "TradFi/Fintech",
    "Synthetics": "Derivatives",
    "SoFi": "TradFi/Fintech",
    "Prediction Market": "Trading",
    "Token Locker": "Yield",
    "Yield Lottery": "Yield",
    "Algo-Stables": "Stablecoins",
    "DEX Aggregator": "Trading",
    "Liquid Restaking": "Restaking/Liquid Restaking",
    "Governance Incentives": "Yield",
    "Restaking": "Restaking/Liquid Restaking",
    "Liquid Staking": "Liquid Staking",
    "Uncollateralized Lending": "Lending",
    "Managed Token Pools": "Trading",
    "Insurance": "TradFi/Fintech",
    "NFT Marketplace": "Trading",
    "NFT Lending": "Lending",
    "Options Vault": "Trading",
    "NftFi": "Trading",
    "Basis Trading": "Trading",
    "Bug Bounty": "TradFi/Fintech",
    "OTC Marketplace": "Trading",
    "Reserve Currency": "Stablecoins",
    "Gaming": "Other",
    "AI Agents": "TradFi/Fintech",
    "Treasury Manager": "TradFi/Fintech",
    "CDP Manager": "Lending",
    "Decentralized Stablecoin": "Stablecoins",
    "Restaked BTC": "Restaking/Liquid Restaking",
    "RWA Lending": "Lending",
    "Staking Pool": "Staking/Liquid Staking",
    "CeDeFi": "TradFi/Fintech",
    "Staking": "Staking/Liquid Staking",
    "Oracle": "Other",
    "Ponzi": "Other",
    "Anchor BTC": "Other",
    "Decentralized BTC": "Other",
    "CEX": "Other",
    "Lending": "Lending"
}


In [4]:
from op_analytics.coreutils.duckdb_inmem.client import init_client
from op_analytics.coreutils.duckdb_inmem.localcopy import dump_local_copy, load_local_copy
from op_analytics.datasources.defillama.dataaccess import DefiLlama

duckdb_client = init_client()

2024-12-17 09:41:24 [info     ] loaded vault from .env file    filename=vault.py lineno=32
2024-12-17 09:41:24 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76


- Pull this data fresh, should be okay to leave protocol metadata date as-is
- I would use "2024-11-30" as your latest date, we ran into a few data issues with more recent data
- Make sure your secrets are up to date, Pedro updated them on Dec 2nd to work with GCS
- There could be lingering data issues but Pedro addressed a bunch today

In [5]:
view1 = DefiLlama.PROTOCOLS_TOKEN_TVL.read(min_date="2023-12-01")

df_protocol_tvl = duckdb_client.sql(
f"""
SELECT
    dt,
    protocol_slug,
    chain,
    token,
    app_token_tvl,
    app_token_tvl_usd
FROM {view1}
""").to_df()

2024-12-17 09:41:25 [info     ] querying markers for 'defillama/protocols_token_tvl_v1' DateFilter(min_date=datetime.date(2023, 12, 1), max_date=None, datevals=None) filename=dailydata.py lineno=107
2024-12-17 09:41:25 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25
2024-12-17 09:41:26 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37
2024-12-17 09:41:26 [info     ] 480 markers found              filename=dailydata.py lineno=121
2024-12-17 09:41:26 [info     ] 383 distinct paths             filename=dailydata.py lineno=127
2024-12-17 09:41:28 [info     ] registered view: 'defillama_protocols_token_tvl_v1' using 383 parquet paths filename=client.py lineno=53
┌──────────────────────────────────┐
│               name               │
│             varchar              │
├──────────────────────────────────┤
│ defillama_protocols_token_tvl_v1 │
└──────────────────────────────────┘



In [8]:
view2 = DefiLlama.PROTOCOLS_METADATA.read(min_date="2024-12-15")

df_metadata = duckdb_client.sql(
f"""
SELECT 
    protocol_name,
    protocol_slug,
    protocol_category,
    parent_protocol,
    CASE WHEN misrepresented_tokens = 'True' THEN 1
        WHEN misrepresented_tokens = 'False' THEN 0
        ELSE 0
    END AS misrepresented_tokens
FROM {view2}
""").to_df()

2024-12-17 09:42:04 [info     ] querying markers for 'defillama/protocols_metadata_v1' DateFilter(min_date=datetime.date(2024, 12, 15), max_date=None, datevals=None) filename=dailydata.py lineno=107
2024-12-17 09:42:04 [info     ] 3 markers found                filename=dailydata.py lineno=121
2024-12-17 09:42:04 [info     ] 3 distinct paths               filename=dailydata.py lineno=127
2024-12-17 09:42:05 [info     ] registered view: 'defillama_protocols_metadata_v1' using 3 parquet paths filename=client.py lineno=53
┌──────────────────────────────────┐
│               name               │
│             varchar              │
├──────────────────────────────────┤
│ defillama_protocols_metadata_v1  │
│ defillama_protocols_token_tvl_v1 │
└──────────────────────────────────┘



In [7]:
## this didn't yield useful results (pun intended)

# YIELD_ENDPOINT = "https://yields.llama.fi/pools"
# session = new_session()
# yield_data = get_data(session, YIELD_ENDPOINT)

# yield_lists = []

# for yield_pool in yield_data["data"]:
#     chain = yield_pool["chain"]
#     project = yield_pool["project"]
#     pool = yield_pool["pool"]
#     symbol = yield_pool["symbol"]
#     underlying_tokens = yield_pool["underlyingTokens"] 
#     yield_lists.append(
#         {
#             "chain_name": chain,
#             "protocol_slug": project,
#             "pool": pool,
#             "symbol": symbol
#             "underlying_tokens": underlying_tokens
#         }
#     )

# df_yield = pd.DataFrame(yield_lists)


In [53]:
# drop duplicates due to an ongoing data upload issue
df_all = pd.merge(
    df_metadata.drop_duplicates(), 
    df_protocol_tvl.drop_duplicates(), 
    on="protocol_slug",
    how="left"
)


In [54]:
# Merge data and join alignment and token categories
df_all = pd.merge(df_all, alignment_df, on="chain", how="left")
df_all["alignment"] = df_all["alignment"].fillna("Other")
df_all = pd.merge(df_all, token_categories, on="token", how="left")
df_all["token_category"] = df_all["token_category"].fillna("Other")


In [55]:
# Chain level misrepresented tokens
df_misrep = (
    df_all[df_all.dt == df_all["dt"].max()-pd.Timedelta(days=1)]
    [["protocol_slug", "chain", "misrepresented_tokens", "token"]]
    .groupby(["protocol_slug", "chain", "misrepresented_tokens"])
    .agg(
        token_count=("token", "nunique"),
        has_usdt=("token", lambda x: 1 if "USDT" in x.values else 0)
    )
    .reset_index()
)

df_misrep["chain_misrepresented_tokens"] = (
    (df_misrep["misrepresented_tokens"] == 1) 
    & (df_misrep["token_count"] == 1) 
    & (df_misrep["has_usdt"] == 1)
).astype(int)

df_all = pd.merge(
    df_all, 
    df_misrep[["protocol_slug", "chain", "chain_misrepresented_tokens"]], 
    on=["protocol_slug", "chain"],
    how="left"
)

In [56]:
# remove protocols and chains

def matches_filter_pattern(s):
    return any(re.search(pattern, s, re.IGNORECASE) for pattern in PATTERNS_TO_FILTER)

df_all["chain"] = df_all["chain"].astype(str)

df_chain_protocol = df_all[["chain", "protocol_slug", "protocol_category"]].drop_duplicates()

df_chain_protocol["protocol_filters"] = (
    df_chain_protocol["chain"].apply(matches_filter_pattern)
    | (df_chain_protocol["protocol_slug"] == "polygon-bridge-&-staking")
    | df_chain_protocol["protocol_slug"].str.endswith("-cex")
    | df_chain_protocol.protocol_category.isin(CATEGORIES_TO_FILTER)
).astype(int)

# small subset for analysis, actual logic will include more (all?) chains
df_chain_protocol["chains_to_keep"] = (
    (df_all.alignment.isin(["OP Chain", "OP Stack Fork"]) 
    | df_all.chain.isin(["Ethereum", "Arbitrum", "Solana", "Polygon", "Sui"]))
    ).astype(int)

filter_mask = (df_chain_protocol.protocol_filters == 0) & (df_chain_protocol.chains_to_keep == 1)

df_filtered = pd.merge(
    df_all,
    df_chain_protocol[filter_mask][["chain", "protocol_slug", "protocol_category"]],
    on=["chain", "protocol_slug", "protocol_category"],
    how="inner",
)



In [57]:
# misc data processing
df_filtered["dt"] = pd.to_datetime(df_filtered["dt"])
df_filtered["parent_protocol"] = df_filtered["parent_protocol"].str.replace("parent#", "")
df_filtered["token"] = df_filtered["token"].str.upper()
df_filtered["token_category"] = df_filtered["token_category"].fillna("Other")

df_filtered["token_category_misrep"] = np.where(
    (df_filtered.chain_misrepresented_tokens == 1),
    "Misrepresented TVL", 
    df_filtered.token_category
)

In [58]:
df_filtered["protocol_category_mapped"] = df_filtered["protocol_category"].map(mapping, na_action="ignore")
df_filtered.loc[df_filtered["protocol_category_mapped"].isna(), "protocol_category_mapped"] = df_filtered["protocol_category"]


In [ ]:
# I used this to find the tokens of interest to manually  map to source protocols

chains = ["Ethereum", "Base", "Mode", "Optimism", "Arbitrum", "Scroll"]

top_tokens = []
for chain in chains:
    df_chain =  df_filtered[
        (df_filtered.dt == "2024-12-01")
        * (df_filtered.chain == chain)
    ]
    top_tvl = df_chain[["token", "app_token_tvl_usd"]].groupby("token").sum().reset_index().sort_values(by="app_token_tvl_usd", ascending=False)

    top_tvl["cumulative_percent"] = (top_tvl["app_token_tvl_usd"].cumsum() / top_tvl["app_token_tvl_usd"].sum()) * 100
    
    # Filter rows where the cumulative percentage is less than or equal to 99%
    top_pct_tvl = top_tvl[top_tvl["cumulative_percent"] <= 95]
    
    # Drop the cumulative_percent column if you no longer need it
    top_pct_tvl = top_pct_tvl.drop(columns=["cumulative_percent"])

    top_tokens += top_tokens.token.to_list()

top_tokens = list(set(top_tokens))

In [83]:
! uv pip install networkx

Using Python 3.12.7 environment at /Users/chuck/codebase/op-analytics/.venv
Audited 1 package in 29ms


In [84]:
import pandas as pd
import networkx as nx
import plotly.express as px

In [674]:
df_source_token = pd.read_csv("source_token_mapping_20241201.csv")

In [734]:
df_filtered_high = df_filtered[
     (df_filtered.chain == "Ethereum")
    & (df_filtered.dt == "2024-12-10")
]

df_merged = pd.merge(
    df_filtered_high, 
    df_source_token, 
    on="token",
    how="inner"
)

In [735]:
df_apps_tvl = (
    df_filtered[
        (df_filtered.chain == "Ethereum")
        & (df_filtered.dt == "2024-12-10")
    ]
    .groupby("parent_protocol")
    .agg(app_tvl_usd=("app_token_tvl_usd", "sum"))
    .reset_index()
)

In [736]:
df_merged = pd.merge(
    df_merged,
    df_apps_tvl.rename(columns={"parent_protocol": "source_protocol", "app_tvl_usd": "source_app_tvl_usd"}),
    on="source_protocol",
    how="left"
)

df_merged = pd.merge(
    df_merged,
    df_apps_tvl.rename(columns={"app_tvl_usd": "parent_app_tvl_usd"}),
    on="parent_protocol",
    how="left"
)

In [737]:
df_app_groups = (
    df_merged
    .groupby(["source_protocol", "parent_protocol", "source_app_tvl_usd", "parent_app_tvl_usd"])
    .agg({"app_token_tvl_usd": "sum"})
    .reset_index()
)

In [744]:
#insane graph chatgpt made for me

import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import community as community_louvain

def create_protocol_network_graph(df):
    # Initialize a directed graph
    G = nx.DiGraph()

    # Add nodes with size attributes (sized by source_app_tvl_usd and parent_app_tvl_usd)
    for _, row in df.iterrows():
        source = row["source_protocol"]
        target = row["parent_protocol"]
        source_tvl = row["source_app_tvl_usd"]
        parent_tvl = row["parent_app_tvl_usd"]
        edge_weight = row["app_token_tvl_usd"]

        # Add source and target nodes with TVL as attributes
        G.add_node(source, tvl=source_tvl)
        G.add_node(target, tvl=parent_tvl)

        # Add edge with 'app_token_tvl_usd' as weight
        G.add_edge(source, target, weight=edge_weight)

    # Detect communities using Louvain method
    partition = community_louvain.best_partition(G.to_undirected())

    # Add community information to each node
    for node, community_id in partition.items():
        G.nodes[node]["community"] = community_id

    # Generate positions using spring layout, adjusting for community clusters
    pos = nx.spring_layout(G, k=0.4, seed=42)  # k controls node spacing

    # Extract edge data for plotting
    edge_x = []
    edge_y = []
    edge_widths = []
    arrows = []

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_widths.append(edge[2]['weight'])

        # Add arrow annotation for the edge
        arrows.append(
            go.layout.Annotation(
                x=x1,
                y=y1,
                ax=x0,
                ay=y0,
                xref="x",
                yref="y",
                axref="x",
                ayref="y",
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=1,
                arrowcolor="#888",
            )
        )

    # Normalize edge widths for better visualization
    max_width = max(edge_widths) if edge_widths else 1
    edge_widths = [w / max_width * 5 + 1 for w in edge_widths]

    # Create edges with arrows
    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color="#888"),
        hoverinfo="none",
        mode="lines",
    )

    # Extract node data for plotting
    node_x = []
    node_y = []
    node_sizes = []
    node_text = []
    node_colors = []

    for node in G.nodes(data=True):
        x, y = pos[node[0]]
        node_x.append(x)
        node_y.append(y)
        node_sizes.append(node[1]["tvl"])
        node_text.append(node[0])
        node_colors.append(partition[node[0]])  # Color nodes based on their community

    # Normalize node sizes for better visualization
    max_node_size = max(node_sizes) if node_sizes else 1
    node_sizes = [s / max_node_size * 50 + 10 for s in node_sizes]

    # Create nodes
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,
        textposition="top center",
        hoverinfo="text",
        marker=dict(
            size=node_sizes,
            color=node_colors,
            colorscale="Viridis",
            line_width=2,
            colorbar=dict(title="Community"),
        ),
    )

    # Create the figure
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Protocol Network Graph with Community Grouping and Directional Arrows",
                        showlegend=False,
                        hovermode="closest",
                        margin=dict(b=0, l=0, r=0, t=0),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        annotations=arrows  # Add arrows to the layout
                    ))

    fig.show()

# # Call the function
create_protocol_network_graph(df_app_groups[df_app_groups.parent_app_tvl_usd >= 1_000_000]
                             )
